In [1]:
!pip install konlpy  # 토큰화에 사용할 konlpy 라이브러리 설치

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 49.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 34.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import os, random
from tqdm.auto import tqdm # 진행도 시각화를 위한 라이브러리

seed=42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)

# 데이터 불러오기

In [3]:
train_data = pd.read_csv("/kaggle/input/2023-ml-project1/nsmc_train.csv", index_col=0)
test_data = pd.read_csv("/kaggle/input/2023-ml-project1/nsmc_test.csv", index_col=0)
print(train_data.shape)
print(test_data.shape)

(149993, 2)
(49999, 1)


In [4]:
x_train = train_data["review"]
y_train = np.array(train_data["rating"])

In [5]:
df=pd.DataFrame(x_train)
df.columns
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)
#df.head(320)
x_train.dtype

dtype('O')

# 자연어 전처리
## \[Empty Module #1\] 데이터 전처리
### 데이터 전처리 수행
> 먼저, 리뷰를 분류하는데 도움이 되거나, 머신러닝 처리에 어려운 단어들을 제거해봅시다.
1. 아래 조건에 맞는 정규표현식을 작성하여 영어와 한글 문자를 제외한 특수문자나 이모지, 숫자 등을 제거해봅시다.
  - <mark>한글 문자(초성 제외), 영어 대문자, 영어 소문자, 띄어쓰기 이외의 문자를 제외</mark>하는 정규표현식 작성
2. 영어 단어의 경우 같은 단어들이 같은 토큰으로 분류될 수 있도록 <mark>대문자로 통일</mark>해줍니다.

In [6]:
##########################################################################################
# Empty Module #1
# 입력: 자연어 상태의 리뷰 텍스트
# 출력: 한글(초성 제외), 영어 대문자, 띄어쓰기로만 구성된 텍스트 
# 입력 예시: "안녕 Hello!!!:)ㅎㅎ반갑다."
# 출력 예시: "안녕 HELLO반갑다"
##########################################################################################
import re

pattern =re.compile('[^가-힣 A-Za-z\s]+')
pattern2=re.compile('[a-z]')


def apply_regex(pattern, text):  # 정규표현식을 이용한 필터링 적용
    text = re.sub(pattern, "", text)  # 정규표현식 패턴에 맞는 값들을 텍스트에서 제거
    # 영어들을 찾아 대문자로 치환하는 코드 작성
    text=re.sub(pattern2,lambda x: x.group(0).upper(),text)
    #[a-z] 패턴은 소문자 x.group(0).upper()는 매칭된 문자열을 대문자로
    return text

x_train_preprocessed = [apply_regex(pattern, str(x[1])) for x in tqdm(x_train.iteritems(), total=len(x_train), desc="pre-processing data")]

/tmp/ipykernel_34/813706970.py:21: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  x_train_preprocessed = [apply_regex(pattern, str(x[1])) for x in tqdm(x_train.iteritems(), total=len(x_train), desc="pre-processing data")]


pre-processing data:   0%|          | 0/149993 [00:00<?, ?it/s]

In [7]:
def apply_regex(pattern, text):  # 정규표현식을 이용한 필터링 적용
    text = re.sub(pattern, "", text)  # 정규표현식 패턴에 맞는 값들을 텍스트에서 제거
    # 영어들을 찾아 대문자로 치환하는 코드 작성
    text=re.sub(pattern2,lambda x: x.group(0).upper(),text)
    
    return text
test_sen='안녕 Hello!!!:)ㅎㅎ반갑다.'
test_obj=np.array([test_sen]).astype('object')

test_proceed = [apply_regex(pattern, str(x[1])) for x in tqdm(np.ndenumerate(test_obj), total=len(test_obj), desc="pre-processing data")]

pre-processing data:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
print(test_proceed)

['안녕 HELLO반갑다']


## \[Empty Module #2\] 단어 토큰화
### Open Korean Text(OKT)를 이용한 단어 토큰화(Tokenization)
- 한국어 자연어 처리 라이브러리인 konlpy의 OKT 래퍼를 통하여 문장을 단어로 토큰화해봅시다.
- 아래 도큐먼트를 참고하여 토큰화를 수행합니다.
  - <mark>이때, OKT 클래스의 특정 매개변수를 이용해 어근화를 진행해줍니다. (도큐먼트 참고)</mark>
- konlpy 도큐먼트: https://konlpy.org/ko/latest/api/konlpy.tag/#okt-class
- OKT 도큐먼트: https://github.com/open-korean-text/open-korean-text

In [9]:
##########################################################################################
# Empty Module #2
# 입력: 자연어 상태의 리뷰 데이터
# 출력: 토큰화와 과정을 거쳐 단어들의 리스트로 변환된 데이터
# 입력 예시: "커피는 역시 학생회관 커피"
# 출력 예시: ["커피", "는", "역시", "학생", "회관", "커피"]
##########################################################################################
from konlpy.tag import Okt
okt = Okt()

def tokenize_words(sentence):
    sentence_tokenized =okt.morphs(sentence,stem=True)
    return sentence_tokenized

In [ ]:
# 약 10-15분 정도 소요됩니다. 
x_train_tokenized = [tokenize_words(x) for x in tqdm(x_train_preprocessed, desc="tokenizing data")]

tokenizing data:   0%|          | 0/149993 [00:00<?, ?it/s]

In [ ]:
print(x_train_tokenized[0])

## \[Empty Module #3\] 불용어 제거
- 조사를 비롯한 불용어들은 많은 횟수 등장하지만, 리뷰의 긍정과 부정 여부를 판단하는데는 도움이 되지 않습니다.
- 데이터에서 아래 리스트로 정의된 불용어들을 제거해줍니다.

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']  #별다른 의미가 없는 불용어들


def exclude_stopwords(text):
    # 위 리스트에 포함된 불용어들을 제거하는 코드 작성
    result=[]
    for w in text:
        if w not in stopwords:
            result.append(w)
   
    return result


x_train_stopwords_excluded = [exclude_stopwords(x) for x in x_train_tokenized]

In [ ]:
print(x_train_stopwords_excluded[1])

## \[Empty Module #4\] 단어 임베딩
### 단어 임베딩 코드 구현
- 토큰화를 거쳐 분리된 단어들을 하나의 정수 값으로 매핑해주는 희소 표현법을 직접 구현해봅시다.
- 입력된 단어가 새로운 단어라면 새로운 정수 값을 할당하고, 이전에 등장한 단어라면 이전에 할당한 정수를 할당하는 함수를 작성합니다.
  - 단, <mark>테스트 데이터에 대해서는 새로운 단어가 등장하면 값을 할당하지 않습니다.</mark>

In [ ]:
##########################################################################################
# Empty Module #4
# 입력: 단어 토큰화된 데이터
# 출력: 임베딩 과정을 거쳐, 각 단어가 하나의 실수 값으로 표현된 데이터
# 입력 예시: ["커피", "역시", "학생", "회관", "커피"]
# 출력 예시: [0, 1, 2, 3, 0]
##########################################################################################

embedding_dict = dict()  # 단어 임베딩을 위한 딕셔너리
embedding_value = 0

def embed_tokens(sentence_tokenized, mode):
    assert mode.upper() in ["TRAIN", "TEST"]
    global embedding_value
    
    sentence_embedded = list()
    for word in sentence_tokenized:
      if mode.upper() == "TRAIN":
        if word not in embedding_dict:
          embedding_dict[word]=embedding_value
          embedding_value+=1
        sentence_embedded.append(embedding_dict[word])
      elif mode.upper() == "TEST":
        if word in embedding_dict:
          sentence_embedded.append(embedding_dict[word])
      
    return sentence_embedded

In [ ]:
# 실행 시간이 제법 소요됩니다. 비정상이 아니니 걱정하지 않으셔도 됩니다.
x_train_embedded = [embed_tokens(x, mode="TRAIN") for x in tqdm(x_train_stopwords_excluded, desc="embedding data")]
print(x_train_embedded[:5])
print("총 %d개의 단어가 임베딩되었습니다."%(embedding_value))

In [ ]:
print(x_train_stopwords_excluded[0])
print(x_train_embedded[0])

print(x_train_stopwords_excluded[1])
print(x_train_embedded[1])

print(x_train_stopwords_excluded[2])
print(x_train_embedded[2])

print(x_train_stopwords_excluded[3])
print(x_train_embedded[3])

print(x_train_stopwords_excluded[4])
print(x_train_embedded[4])

## \[Empty Module #5\] 문장 벡터화
### Bag of Words 방법을 사용한 문장 벡터화
- 캐글 프로젝트 설명 페이지의 Bag of Words 방법 설명을 참고하여 Bag of Words 방법을 직접 구현해봅시다.

In [ ]:
##########################################################################################
# Empty Module #5
# 입력: 임베딩 과정을 거친 데이터
# 출력: BoW 형태로 변환되어, M차원의 고정된 크기를 가진 벡터로 변환된 데이터
# 힌트: np.zeros((2, 3))는 [2, 3] 크기의 0으로 가득 찬 행렬을 생성합니다.
##########################################################################################
M = 45361
def to_BoW_representation(x):
    shape = (len(x),M)# BoW는 어떤 shape를 가져야 할까요?
    x_BoW = np.zeros(shape)
    for i in tqdm(range(len(x)), desc="making BoW representation"):
        for word in x[i]:#현재문장의 단어마다
            word_idx=(word)
            x_BoW[i][word_idx]+=1#해당 단어 인덱스 위치 값 1 증가
    return x_BoW
x_train_BoW = to_BoW_representation(x_train_embedded)

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)
#x_train_BoW[1]
#print(np.unique(x_train_BoW[0]))
x_train_BoW.shape

## \[Empty Module #6\] 차원 축소
- 우리가 만든 BoW는 수많은 리뷰에 등장하는 모든 단어들을 사용하여 만들어졌기 때문에, 엄청난 양의 단어들을 가지고 있습니다.
- 그러나, 실제로 영화 리뷰에 쓰이는 단어들은 이보다 적기 때문에, 많은 단어들이 전체 데이터에서 실제로는 한번도 등장하지 않거나, 매우 조금 등장하면서 공간을 차지하고 있을 것 입니다.
- 데이터의 크기를 줄여 머신러닝 모델이 중요한 정보에 집중할 수 있도록 해봅시다.
- <mark>학습 데이터에서 50번 미만으로 등장한 단어들을 제외</mark>해줍니다.

In [ ]:
##########################################################################################
# Empty Module #6
# 입력: BoW 형태로 변환된 (N, M) 크기의 데이터
# 출력: 등장 빈도가 적은 단어들을 제외한 (N, m) 크기의 더 작은 데이터
##########################################################################################

In [ ]:
# 힌트
# 1. 먼저 전체 데이터에서 각 단어가 등장한 횟수를 세어보세요.
# 2. 그 다음, 등장 횟수가 50회 미만인 단어들을 찾습니다.
# 3. 해당 단어들을 데이터에서 제거하는 코드를 작성합니다.
# 4. 설계를 잘 하고 구현을 시작해야 어렵지 않습니다.

In [ ]:
#cnt=np.sum(x_train_BoW,axis=0)
#print(np.unique(cnt))

word_freq = np.sum(x_train_BoW,axis=0 )# 각 단어가 등장한 횟수를 세어줍니다.
#print(word_freq[0:5])
to_remove_idx = np.where(word_freq < 50) # 등장 횟수가 50회 미만인 단어들의 인덱스를 찾습니다.

#print(to_remove_idx[0:5])

x_train_BoW_reduced=np.delete(x_train_BoW,to_remove_idx,axis=1)
#num_arr=np.asarray(to_remove)
#print(num_arr.shape)
#x_train_BoW_reduced = np.delete(x_train_BoW, to_remove, axis=1) # 해당 인덱스에 해당하는 열(단어)을 제거합니다.
        
    

print("원본 BoW 크기:", x_train_BoW.shape)
print("차원 축소 후 크기:", x_train_BoW_reduced.shape)

## \[Empty Module #7\]  분류 수행 및 제출: Bag of Words
- 이제 모든 문장이 고정된 크기 $m$ 차원의 벡터로 변환되었습니다.
- 원하는 모델을 사용하여, 각 문장의 영화에 대한 긍정적인 리뷰인지, 부정적인 리뷰인지 분류해봅시다.(Baseline 모델은 로지스틱 회귀입니다)
- 그 다음, 결과를 기록하여 Kaggle에 제출해봅시다!

In [ ]:
##########################################################################################
# Empty Module #7
# 지금까지 전처리한 데이터로 분류를 수행하여, kaggle에 제출해봅시다.
# Baseline은 로지스틱 회귀입니다.
##########################################################################################
# 분류기 정의 및 학습 수행 코드 작성



In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_val,Y_train,Y_val=train_test_split(x_train_BoW_reduced,y_train,test_size=0.2,stratify=y_train,random_state=42)

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

C=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]

for i in range(0,10):
    logi=LogisticRegression(max_iter=500,random_state=42,n_jobs=-1,C=C[i])
    logi.fit(X_train,Y_train)
    pred=logi.predict(X_val)
    print(accuracy_score(Y_val,pred))


In [ ]:
logi=LogisticRegression(max_iter=500,random_state=42,n_jobs=-1,C=0.6)
logi.fit(x_train_BoW_reduced,y_train)

In [ ]:
# TEST 데이터를 전처리
x_test = test_data["review"]
x_test_preprocessed = [apply_regex(pattern, str(x[1])) for x in tqdm(x_test.iteritems(), total=len(x_test), desc="pre-processing data")]
x_test_tokenized = [tokenize_words(x) for x in tqdm(x_test_preprocessed, desc="tokenizing data")]
x_test_stopwords_excluded = [exclude_stopwords(x) for x in x_test_tokenized]
x_test_embedded = [embed_tokens(x, mode="TEST") for x in tqdm(x_test_stopwords_excluded, desc="embedding data")]

x_test_BoW = to_BoW_representation(x_test_embedded)

word_freq = np.sum(x_train_BoW,axis=0 )# 각 단어가 등장한 횟수를 세어줍니다.
to_remove_idx = np.where(word_freq < 50) # 등장 횟수가 50회 미만인 단어들의 인덱스를 찾습니다.
x_test_BoW_reduced=np.delete(x_test_BoW,to_remove_idx,axis=1)

In [ ]:
x_test_BoW_reduced.shape

In [ ]:
# TEST 데이터에 대한 예측 수행 코드 작성

In [ ]:
submit = pd.read_csv("/kaggle/input/2023-ml-project1/sample_submission.csv")
# TEST 데이터에 대한 예측 값을 csv로 저장하는 코드 작성

In [ ]:
pred = logi.predict(x_test_BoW_reduced)
submit['rating']=pred
submit.to_csv("submit_logi.csv", index=False)

## \[Empty Module #8\] TF-IDF 적용
- BoW에서는 고려하지 않는 각 단어들의 중요도를 고려하기 위해, TF-IDF를 적용해봅시다.
- 캐글 프로젝트 설명 페이지의 설명을 참고하여 빈칸을 채워, TF-IDF를 구현해봅시다.

In [ ]:
print(embedding_dict['배우'])


In [ ]:
word=list(embedding_dict.keys())
print(len(word))

In [ ]:
print(x_train_BoW_reduced.shape)

In [ ]:
##########################################################################################
# Empty Module #8
# 빈칸을 적절히 채워넣어 TF-IDF를 위한 Inverse Document Frequency를 계산해봅시다.
##########################################################################################
from math import log
N = len(x_train_BoW)  # 총 데이터 샘플의 수

def tf(t, d):
    return d.count(t)

result=[]

def calculate_document_frequency(x):
    result=(np.sum(x,axis=0))
    return result 
        
                
#calculate_document_frequency(x_train_BoW_reduced)        

    
IDF=[]
def calculate_inverse_document_frequency(document_frequency):
    for i in range(0,3127):
        cnt=document_frequency[i]
        print(cnt)
        IDF.append(log(N/(cnt+1)))
    return IDF

   
     
    
document_frequency=calculate_document_frequency(x_train_BoW_reduced)

inverse_document_frequency =calculate_inverse_document_frequency(document_frequency)   

In [ ]:
x_train_tfidf = x_train_BoW_reduced*inverse_document_frequency

## \[Empty Module #9\]  분류 수행 및 제출: TF-IDF
- TF-IDF를 적용한 결과를 기록하여 Kaggle에 제출해봅시다!

In [ ]:
##########################################################################################
# Empty Module #9
# TEST 데이터에 TF-IDF를 적용하여 모델을 학습, 예측을 수행하고 kaggle에 제출해봅시다.
# 이때, BoW와 같은 모델을 사용하여 성능을 비교해봅시다.
##########################################################################################
# 분류기 정의 및 학습 수행 코드 작성

logi=LogisticRegression(max_iter=500,random_state=42,n_jobs=-1,C=0.6)
logi.fit(x_train_tfidf,y_train)

In [ ]:
# TEST 데이터를 전처리하는 코드 작성
x_test_tfidf = x_test_BoW_reduced*inverse_document_frequency

In [ ]:
# 예측을 수행하는 코드 작성
pred = logi.predict(x_test_tfidf)

In [ ]:
submit = pd.read_csv("/kaggle/input/2023-ml-project1/sample_submission.csv")
# TEST 데이터에 대한 예측 값을 csv로 저장하는 코드 작성
submit['rating']=pred
submit.to_csv("submit_logi_tfidf.csv", index=False)

## 결과 비교 TIP
- BoW와 TF-IDF에 같은 모델을 적용하여 성능의 차이를 비교해봅시다.
- 각각의 방법론에 여러가지 모델을 적용하며, 사용한 방법에 따라 더 적절한 모델이 있는지 고려해봅시다.
- 실험 결과들을 토대로, 왜 이런 결과가 도출되었는지 고민해봅시다.